# config

The Midgard **config** module can be used for handling of configuration read from a configuration file. The Midgard **config** module uses the standard library module **configparser** for reading configuration files. The configuration file structure is similar to the standard Microsoft Windows INI files. A configuration file consists of one or several sections, which are indicted with square brackets. Each configuration section consists of one or more entries and each configuration entry consists of a key and a value. A configuration file example:

    [section_1]
    option_1 = value  # 1st entry
    option_2 = value  # 2nd entry
    
    [section_2]
    option_3 = value
    option_4 = value
    
Midgard provides modules for file and analysis configuration. The file configuration specifies the location of input and output files used in an analysis. The analysis configuration defines the properties of the analysis, that means for example what kind of models are used or results are written.

The file configuration files can include several sections, which are
called in this case file keys. Each file key section describes the location of the input or output file and optionally it can include a short description, the format specification source, the URL from where a input file can be downloaded. In the following the file configuration keys are shown, which a file key section can include:

    [<file key>]
    filename = <file name>
    aliases = <file name>, <file name>, ...
    directory = <directory name>
    publish = <path>
    description = <description>
    specification = <url>
    origin = <url>
    url = <url>

## Use config module

The basis of the config module is the **Configuration** class. The table below describes shortly the exiting **Configuration** methods:

| Method           | Description                                                           |
| :--------------- | :-------------------------------------------------------------------- |
| as_dict          | The configuration represented as a dictionary | 
| as_str           | The configuration represented as a string |
| clear            | Clear the configuration |
| clear_vars       | Clear the configuration variables |
| exists           | Check if a configuration entry exists |
| get              | Get an entry from a configuration with possibility for override and default value |
| profiles         | List of profiles currently being used in Configuration |
| read_from_file   | Read a configuration from one or more files |
| section_names    | Names of sections in Configuration |
| sections         | Sections in Configuration |
| sources          | Sources of entries in Configuration |
| update           | Update a configuration section with a configuration entry |
| update_from_config_section | |
| update_from_dict | |
| update_from_file | Update the configuration from a configuration file |
| update_from_options | |
| update_on_file   | Context manager for updating a configuration on file |
| update_vars      | Update the configuration variables |
| vars             | The configuration variables |
| write_to_file    | Write the configuration to a file |

Following analysis configuration file are read as an example:

    [gnss]
    apriori_orbit            = broadcast
    navigation_message_type  = G:LNAV E:FNAV
    removers                 = ignore_satellite, ignore_unhealthy_satellite, gnss_clean_orbit
    removers:add_sections
    stations                 = onsa, mets, nya1, tro1
    max_iteration            = 2

    [ignore_satellite]
    satellites                = E14 E18 E20 C05

It will be explained how to use a **config** module for reading the given analysis configuration file. The first step is to generate an instance of the **Configuration** class. An example:

In [ ]:
# Import Configuration class
from midgard.config.config import Configuration

# Get "empty" Configuration instance
cfg = Configuration(name='config')

# Get Configuration instance by reading
#     one configuration file:
cfg = Configuration.read_from_file('config', './examples/config/config.conf')
#     or several configuration files:
cfg = Configuration.read_from_file('config', 
          ['./examples/config/config.conf', './examples/config/config_local.conf']
      )

### Update configuration
The configuration instance can be updated like:

In [ ]:
# Update configuration by defining section and option name (key)
cfg.update(section="gnss", key="max_iteration", value=5)

# Upate configuration by reading configuration file
cfg.update_from_file(file_path='./examples/config/config.conf')

### Access configuration
The configuration entries can be accessed as follows:

    <instance_name>.<section_name>.<entry_name>

That means the instance name, the section name and at the end the entry name has to be defined, e.g. like `cfg.gnss.max_iteration`. If the configuration is accessed like that, than an instance of the **ConfigurationEntry** class is returned. The ConfigurationEntry instance has several access methods that convert the entry to a given
data type. Following data types are defined:
    
    bool:      boolean type
    date:      Date object
    datetime:  Datetime object
    dict:      dictionary collection
    float:     float type
    int:       integer type
    list:      list collection
    path:      PosixPath object
    str:       string type
    tuple:     tuple sequence 

The data type has to be added to the given access statement above:

    <instance_name>.<section_name>.<entry_name>.<data_type>

Examples:

In [ ]:
# Integer data type
cfg.gnss.max_iteration.int

In [ ]:
# String data type
cfg.gnss.max_iteration.str

In [ ]:
# Dictionary
cfg.gnss.max_iteration.dict

In addition **get()** routine can be used for getting configuration file entries. The **get()** routine can also be applied for defining default values for an entry. Example:

In [ ]:
# Access entry via get
cfg.get(section="gnss", key="max_iteration")
cfg.gnss.get(key="max_iteration")

In [ ]:
# Access entry via get by defining data type
cfg.gnss.get(key="max_iteration").int

In [ ]:
# Define default entry, if belonging option has no defined value in 
# configuration file
cfg.gnss.get(key="max_iteration", default=2).int

It can be checked with routine **exists()**, if an option is defined in configuration file or not. Example:

In [ ]:
# Check if option 'max_iteration' exists in section 'gnss'
cfg.gnss.exists(key="max_iteration")

### Additional config functionality
TODO: Description of profiles, add_section, master_section and fallback has to be described.

#### Master section
**master_section** could also be named **default_section**. **master_section** is used, if no section name is defined. That means instead of accessing configuration like:
    
        <instance_name>.<section_name>.<entry_name>.<data_type>
        
it could be done like that:    

        <instance_name>.<entry_name>.<data_type>
        
For example **Where** uses the master section to access common configuration, which is valid for all pipelines like VLBI, SLR or GNSS. In this case the pipeline name has not to be specified explicity by accessing the configuration. For example configuration entry `config.tech.ephemerides` can be accessed independently of used pipeline in `apriori.ephemerides`, because `ephemerides` entry is defined in master section `all` for all pipelines.

An example:

In [ ]:
# Define master section
cfg.master_section = "gnss"

# Access 'max_iteration' entry in section 'gnss' (instead of using cfg.gnss.max_iteration.int)
cfg.max_iteration.int

## Use config files module

The basis of the config files module is the **FileConfiguration** class. The table below describes shortly the exiting **FileConfiguration** methods:

| Method           | Description                                                           |
| :--------------- | :-------------------------------------------------------------------- |
| download_file    | Download a file from the web and save it to disk |
| empty_file       | Check if a file is empty |
| encoding         | Look up the encoding for a given file key |
| glob_paths       | Find all filepaths matching a filename pattern |
| glob_variable    | Find all possible values of variable |
| is_path_zipped   | Indicate whether a path is to a gzipped file or not |
| open             | Open a file based on information in a configuration (file key needed) |
| open_path        | Open a file with given path |
| path             | Construct a configured filepath (file key needed) for a given file with variables  |
| url              | Construct a URL for a given file with variables |

Hereby should be noted, that the **FileConfiguration** class inherits from the **Configuration** class, which means that the methods mentioned in Section *Use config module* above can also be used.

Following file configuration file are read as an example:

    [rinex_obs]
    filename        = {station}{doy}0.{yy}o.Z
    aliases         = {STATION}00NOR_R_{yyyy}{doy}0000_01D_30s_MO.rnx{gz}
    directory       = {path_data}/gnss/obs/{yyyy}/{doy}
    description     = GNSS observations in RINEX format version 2.11
    specification   = ftp://ftp.igs.org/pub/data/format/data/format/rinex211.txt
    origin          = ftp://cddis.gsfc.nasa.gov/pub/gps/data/daily/{yyyy}/{doy}/{yy}o/
    url             = ftp://cddis.gsfc.nasa.gov/pub/gps/data/daily/{yyyy}/{doy}/{yy}o/

It will be explained how to use a **config.files** module for reading this file configuration file. The first step is to generate an instance of the **FileConfiguration** class. An example:

In [ ]:
# Import FileConfiguration class
from midgard.config.files import FileConfiguration

# Get FileConfiguration instance by reading file configuration file
files = FileConfiguration.read_from_file('files', './examples/config/files.conf')

# Define file variables for replacement of {station}, {yyyy}, {yy} and {doy} placeholders
file_vars=dict(station='tro1', yyyy='2019', yy='19', doy='001')

# Get file path for file key 'rinex_obs'
files.path("rinex_obs", file_vars=file_vars)

# Get RINEX observation file description
files.rinex_obs.description.str